# Get_Template: 
This script constructs a very simple `template` of a miyake event. The sinusoidal component of the production function is ignored resulting in a smooth curve. The template is constructed using the best fitting `model`-`dataset` pair  from the 774AD year. `emcee` is used to fit the parameters of the production function which is then used to generate the `.csv` file containing the `dc14` concentrations over just the event.

In [47]:
from os import walk, getcwd
from ticktack.fitting import SingleFitter
from ticktack import load_presaved_model
from jax.numpy import array, pi, mean

In [49]:
data_sets = []  # List for storing the data set locations
data_sets_directory = f"{getcwd()}/datasets/775AD" # Home directory of the data 
for (root, dirs, files) in walk(data_sets_directory):    # Looping over directories 
    for file in files:  # Looping through the files 
        file_path = root + "/" + file   # Setting up the path 
        data_sets.append(file_path)  # Extending the stored directoriess

In [ ]:
carbon_box_model = load_presaved_model(     # Generating the CarbobBoxModel using ticktack
    "Guttler14",                            # Selecting an arbitrary model 
    production_rate_units="atoms/cm^2/s"    # Units of the model
)

bayesian_model = SingleFitter(carbon_box_model)   # Fitting a model 
bayesian_model.prepare_function(model="simple_sinusoid") # Generating the simple sin model
bayesian_model.load_data(data_sets[8])   # Loading the data into the model  

In [ ]:
mcmc_samples = bayesian_model.MarkovChainSampler(
    array([775., 1./12, pi/2., 81./12]), 
    likelihood=bayesian_model.log_joint_simple_sinusoid,
    args=(array([770., 0., -pi, 0.]), array([780., 5., pi, 15.])))
mean(mcmc_samples, axis=0)

In [58]:
template = get_model("Guttler14", data_sets[8])
parameters = get_production_function(template)

Running burn-in...


100%|██████████| 500/500 [01:06<00:00,  7.50it/s]


Running production...


100%|██████████| 1000/1000 [02:10<00:00,  7.64it/s]


So I now need to think about extracting the internals of the functions. I might keep them for now, but I dislike it.

In [57]:
template.dc14(parameters)

DeviceArray([773.63991719,   0.81313909,   2.39273268,   6.01649158], dtype=float64)